**Name:** Xian Jia Le ,Ben

**EID:** 56214537

**Kaggle Team Name:** Glanceman

# CS5489 - Assignment 2 - Urban Sound Tagging
Due date: see Assignment 2 on Canvas

# FINAL SOLUTION

In [1]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import csv
from scipy import io
import pickle
from IPython.display import Audio, display

In [2]:
def showAudio(info):
    myfile = 'musicmp3/' + info['fname'] + '.mp3'
    if os.path.exists(myfile):
        display(Audio(myfile))
    else:
        print("*** mp3 file " + myfile + " could not be found ***")

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out

In [3]:
train_tags  = load_pickle('sounddata/train_tags.pickle3')
train_mfccs = load_pickle('sounddata/train_mfccs.pickle3')
train_mels  = load_pickle('sounddata/train_mels.pickle3')
train_info  = load_pickle('sounddata/train_info.pickle3')

test_mfccs = load_pickle('sounddata/test_mfccs.pickle3')
test_mels  = load_pickle('sounddata/test_mels.pickle3')
test_info  = load_pickle('sounddata/test_info.pickle3')

In [4]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [5]:
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs  = compute_delta_mfccs(test_mfccs)

In [6]:
alltagnames, alltagnames_counts = unique(concatenate(train_tags), return_counts=True)
for a,b in zip(alltagnames, alltagnames_counts):
    print("{}: {}".format(a, b))

# remove some tags
blacklist = ['2-3_hoe-ram_presence', '2-4_pile-driver_presence', '5-2_car-alarm_presence', '5-X_other-unknown-alert-signal_presence', '6-3_ice-cream-truck_presence', '7-3_large-crowd_presence', '7-X_other-unknown-human-voice_presence']
tagnames = [t for t in alltagnames if t not in blacklist]
print(tagnames)

1-1_small-sounding-engine_presence: 193
1-2_medium-sounding-engine_presence: 616
1-3_large-sounding-engine_presence: 775
1-X_engine-of-uncertain-size_presence: 307
1_engine_presence: 1492
2-1_rock-drill_presence: 264
2-2_jackhammer_presence: 179
2-3_hoe-ram_presence: 79
2-4_pile-driver_presence: 55
2-X_other-unknown-impact-machinery_presence: 255
2_machinery-impact_presence: 650
3-1_non-machinery-impact_presence: 349
3_non-machinery-impact_presence: 349
4-1_chainsaw_presence: 101
4-2_small-medium-rotating-saw_presence: 100
4-3_large-rotating-saw_presence: 96
4-X_other-unknown-powered-saw_presence: 79
4_powered-saw_presence: 267
5-1_car-horn_presence: 286
5-2_car-alarm_presence: 28
5-3_siren_presence: 174
5-4_reverse-beeper_presence: 175
5-X_other-unknown-alert-signal_presence: 70
5_alert-signal_presence: 635
6-1_stationary-music_presence: 84
6-2_mobile-music_presence: 26
6-3_ice-cream-truck_presence: 6
6-X_music-from-uncertain-source_presence: 73
6_music_presence: 150
7-1_person-or-sma

In [7]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [8]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes = tags2class(train_tags, tagnames)

In [9]:
import csv

def write_csv_kaggle_tags(fname, tagnames, Yscores):
    # header
    tmp = [['Id']]
    for t in tagnames:
        tmp[0].append(t)    
    
    # add ID numbers for each Y, and usage if necessary
    for i in range(len(Yscores)):
        tmp2 = [(i+1)]
        for t in range(len(tagnames)):
            tmp2.append(Yscores[i,t])
        
        tmp.append(tmp2)
        
    # write CSV file
    f = open(fname, 'w')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

## YOUR CODE and DOCUMENTATION HERE

In [10]:
from sklearn import linear_model,cluster
from sklearn import feature_extraction

def bow_transform(model, mfccs, num_components=100):
    numwords = num_components #100
    bows = zeros((len(mfccs), numwords))
    for i in range(len(mfccs)): # loop tracks
        w = model.predict(mfccs[i]) # predict track class 
        bw = bincount(w, minlength=numwords) # count num wthin first 100
        bows[i,:] = bw # append the count for 2794
    return bows

def logistic_train_predict(trainX,trainY,testX,fileName):
    tagmodels_log = {}
    for i,t in enumerate(tagnames): #loop tag name
        print('training {} - {}'.format(i, t))
        myY = trainY[:,i].ravel() # get the corrspoend class col , flatten without copy
        lr = linear_model.LogisticRegressionCV(Cs=logspace(-4,4,20),max_iter=1000, cv=5, class_weight='balanced', solver='liblinear')
        lr.fit(trainX, myY)
        tagmodels_log[t] = lr #store tag classifer

    # # predict train
    # train_predscore = zeros(shape=(len(train_mfccs), len(tagnames)))

    # for i,t in enumerate(tagnames): # do prediction for each class 
    #     print('predicting {} - {}'.format(i, t))

    #     tmp = tagmodels_log[t].decision_function(trainX)
    #     train_predscore[:,i] = tmp

    # plot_roc(tagnames, trainY, train_predscore) #test with training data

    test_predscore = zeros(shape=(len(test_mfccs), len(tagnames)))

    for i,t in enumerate(tagnames):
        print('predicting {} - {}'.format(i, t))

        tmp = tagmodels_log[t].decision_function(testX)
        test_predscore[:,i] = tmp

    # write result 
    write_csv_kaggle_tags("output/"+fileName+".csv", tagnames, test_predscore)

**Feature Engineer**

In [11]:
def compute_delta_mfccs2(mfccs):
    dmfccs = []
    for m in mfccs: # loop audio audio
        tmpPrev =m[:-1]-m[1:]
        tmpNext = m[1:] - m[0:-1] # subtract the previous one -> get variation over the time (assume time 0 is 0)
        dm = hstack((tmpPrev[:-1], m[1:-1], tmpNext[1:])) #first 20 is original freq value || 21 is variation for next 
        dmfccs.append(dm)
    return dmfccs
train_dmfccs2 = compute_delta_mfccs2(train_mfccs)
test_dmfccs2  = compute_delta_mfccs2(test_mfccs)
all_dmfccs2 = vstack(train_dmfccs2)
print(all_dmfccs2.shape)

(1198613, 60)


**Classification** kmeans + logistic 

In [12]:


km = cluster.KMeans(n_clusters=124, random_state=4487)
km.fit(all_dmfccs2[0::100])  # subsample by 10 to make it faster
km.cluster_centers_
print(f"number of cluster {km.cluster_centers_.shape}")
train_km_bow = bow_transform(km, train_dmfccs2,km.cluster_centers_.shape[0])
test_km_bow  = bow_transform(km, test_dmfccs2,km.cluster_centers_.shape[0])

print(f"shape = {train_km_bow.shape}")
print(f"shape = {train_km_bow.shape}")

tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
train_km_Xtf = tf_trans.fit_transform(train_km_bow )
test_km_Xtf  = tf_trans.transform(test_km_bow )

#preidcition
logistic_train_predict(trainX=train_km_Xtf,trainY=train_classes,testX=test_km_Xtf,fileName="sound_KM+Logistic_Final")

d:\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


number of cluster (124, 60)
shape = (2794, 124)
shape = (2794, 124)
training 0 - 1-1_small-sounding-engine_presence
training 1 - 1-2_medium-sounding-engine_presence
training 2 - 1-3_large-sounding-engine_presence
training 3 - 1-X_engine-of-uncertain-size_presence
training 4 - 1_engine_presence
training 5 - 2-1_rock-drill_presence
training 6 - 2-2_jackhammer_presence
training 7 - 2-X_other-unknown-impact-machinery_presence
training 8 - 2_machinery-impact_presence
training 9 - 3-1_non-machinery-impact_presence
training 10 - 3_non-machinery-impact_presence
training 11 - 4-1_chainsaw_presence
training 12 - 4-2_small-medium-rotating-saw_presence
training 13 - 4-3_large-rotating-saw_presence
training 14 - 4-X_other-unknown-powered-saw_presence
training 15 - 4_powered-saw_presence
training 16 - 5-1_car-horn_presence
training 17 - 5-3_siren_presence
training 18 - 5-4_reverse-beeper_presence
training 19 - 5_alert-signal_presence
training 20 - 6-1_stationary-music_presence
training 21 - 6-2_mobi